# **Importando bibliotecas**

> Primeiro de tudo, todas as bibliotecas utilizadas na implementação são importadas.



In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import multivariate_normal


  # **1 - Preparação dos dados**

  > Nesta seção são realizadas várias etapas comuns no pré-processamento e na preparação de dados para treinar e avaliar um modelo de classificação, sendo elas o leitura e renomeação dos dados, limpeza e conversão de dados categóricos, preparação dos dados para modelagem e divisão dos dados em conjunto de treino e teste.
  
  >❗A divisão em conjuntos de treino e teste é especialmente importante para avaliar a capacidade do modelo de generalizar para novos dados não vistos durante o treinamento.



  

## Leitura e renomeação do conjunto de dados

In [2]:
# Carrega o conjunto de dados
df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data")

# Renomeia as colunas do DataFrame
df.columns = ["age","workclass","finalweight","education",
                 "education-num","martial-status","occupation",
                 "relationship","race","sex","capital-gain",
                 "capital-loss","hours-per-week ","native-country",
                 "income"]


## Limpeza e preparação dos dados

In [3]:
# Limpeza nos dados removendo espaços em branco desnecessários do início e do final das strings

for col in df.columns:
  if df[col].dtype == object:
      df[col] = df[col].str.strip()

X = df.drop('income', axis=1)
y = df.income

X.head()

le = LabelEncoder()
le.fit(['<=50K', '>50K'])
print(list(le.classes_))

# Converte variáveis categóricas (strings) em representações numéricas para preparar os dados categóricos para o modelo de classificação

atributos = X.select_dtypes(include=['object']).columns

for atributo in atributos:
  X[atributo] = le.fit_transform(X[atributo])

['<=50K', '>50K']


## Divisão do conjunto de dados (treino/teste)

In [4]:
train_ratio = 0.8   # Proporção 80/20 para treino e teste respectivamente

# Divide os dados em conjunto de treino e teste
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, random_state=42)

# Obtem as dimensões do conjunto de teste
dimensoes_teste = x_test.shape

# Imprime as dimensões do conjunto de teste
print(dimensoes_teste)

(6512, 14)


# **2 - Criação do modelo**





> Nesta seção é realizado o treinamento de um modelo probabilístico utilizando dados de treinamento para estimar as distribuições de probabilidade para cada classe e em seguida são utilizados tais modelos para fazer predições sobre novos dados (conjunto de teste), determinando a classe mais provável para cada elemento com base nas probabilidades condicionais calculadas.






## **Classificação do dados**


In [ ]:

# Obtém as classes únicas do último atributo (coluna) do DataFrame df
classes_df = np.array(pd.unique(df[df.columns[-1]]), dtype=str)

# Cria uma matriz de probabilidades inicializada com zeros para armazenar as probabilidades de pertencer a cada classe para os elementos do conjunto de teste
matriz_probabilidades = pd.DataFrame(data=np.zeros((x_test.shape[0], len(classes_df))), columns=classes_df)

# Cria um array de zeros para armazenar a probabilidade de pertencer a cada classe
matriz_c = np.zeros(len(classes_df))

# Calcula a probabilidade de pertencer a cada classe e estima os parâmetros de distribuição para cada classe
for i in np.arange(0, len(classes_df)):
    # Seleciona os índices dos elementos no conjunto de treinamento que pertencem à classe i
    itens = tuple(np.where(y_train == classes_df[i]))

    # Calcula a probabilidade de pertencer à classe i
    matriz_c[i] = len(itens) / len(y_train)

    # Seleciona os elementos da classe i do conjunto de treinamento
    a = x_train.iloc[itens]

    # Calcula a média e a matriz de covariância para a classe i
    media = np.mean(a, axis=0)
    covariancia = np.cov(np.transpose(a))

    # Calcula a densidade de probabilidade multivariada para cada elemento do conjunto de teste
    for j in np.arange(0, x_test.shape[0]):
        # Seleciona a linha j do DataFrame x_test e atribui-a à variável x
        x = x_test.iloc[j]

        # Calcula a densidade de probabilidade para o ponto x pertencendo à classe i
        pj = multivariate_normal.pdf(x, mean=media, cov=covariancia, allow_singular=True)

        # Armazena a probabilidade calculada na matriz de probabilidades para a classe i e o elemento j do conjunto de teste
        matriz_probabilidades[classes_df[i]][j] = pj * matriz_c[i]

# Imprime a matriz de probabilidades resultante
print(matriz_probabilidades)



## **Utilizando o modelo de classificação**

In [6]:
predicao = []

# Itera sobre cada linha do conjunto de teste
for i in np.arange(0, x_test.shape[0]):
    # Encontra o índice da coluna com a maior probabilidade para a amostra i
    c = np.argmax(np.array(matriz_probabilidades.iloc[[i]]))

    # Adiciona a classe correspondente ao índice "c" à lista de predições
    predicao.append(classes_df[c])

# Converte a lista de predições para um array numpy com dtype=str, que contém as classes previstas para cada amostra do conjunto de teste
predicao = np.array(predicao, dtype=str)

# Imprime as predições para todas as amostras do conjunto de teste
print(predicao)

['<=50K' '<=50K' '<=50K' ... '<=50K' '<=50K' '>50K']


# **3 - Avaliação do modelo**




> Nesta seção há a finalização do processo de avaliação do modelo, fornecendo uma métrica quantitativa (acurácia) que indica o quão bem o modelo está performando em fazer predições corretas sobre o conjunto de testes.


> ❗Interpretação da Precisão (Acurácia):

*   A acurácia é uma métrica comum utilizada para avaliar a performance de um modelo de classificação.
*   Ela é calculada como a proporção de predições corretas em relação ao total de amostras no conjunto de teste.
*   Valores mais altos de acurácia indicam um melhor desempenho do modelo em fazer predições corretas.








## Medir precisão

In [7]:
# Calcula a precisão das predições comparadas aos rótulos reais
precisao = accuracy_score(predicao, y_test)

## Para que proporção dos registros contidos no conjunto de teste o seu modelo consegue predizer corretamente a classe?

In [8]:
print(f'Para aproximadamente {precisao * 100:.2f}% das amostras no conjunto de teste, o modelo fez a predição correta da classe')

Para aproximadamente 78.01% das amostras no conjunto de teste, o modelo fez a predição correta da classe
